In [26]:
from taxcalc import *
from taxcalc.utils import *
from bokeh.io import show, output_notebook
from bokeh.charts import Bar, Scatter
from bokeh.layouts import column
from collections import OrderedDict
import copy
from notebookfunctions import distribution, index_list
output_notebook()

Loading BokehJS ...

### Missing Variables

In [27]:
cps = pd.read_csv('../Dropbox/cps.csv')
usable = open('taxcalc/records_variables.json')
usable_vars = json.load(usable)
usable.close()
missing = 0
for item in usable_vars['read'].keys():
    if item not in cps.columns:
        print '{}: {}'.format(item, usable_vars['read'][item]['desc'])
        missing += 1
print '\nTotal Variables Missing: {}'.format(missing)

p23250: Sch D: Net long-term capital gains/losses
p25470: Sch E: Royalty depletion and/or rental depreciation
e09800: Unreported payroll taxes from Form 4137 or 8919
e02000: Sch E rental, royalty, S-corp, etc, income/loss
e62900: Alternative Minimum Tax foreign tax credit from Form 6251
p08000: Other tax credits (but not including Sch R credit)
e58990: Investment income elected amount from Form 4952
e00700: Taxable refunds of state and local income taxes
e03290: Health savings account deduction from Form 8889
e07240: Retirement savings contributions credit from Form 8880
e19200: Sch A: Interest paid
e27200: Sch E: Farm rent net income or loss
e01200: Other net gain/loss from Form 4797
e03500: Alimony paid
e07260: Residential energy credit from Form 5695
blind_spouse: 1 if spouse is blind; otherwise 0
p22250: Sch D: Net short-term capital gains/losses
e03220: Educator expenses
e07400: General business credit from Form 3800
f6251: 1 if Form 6251 (AMT) attached to return; otherwise 0
blin

In [28]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items

In [29]:
# Create calculator from the PUF
calc_puf = Calculator(records=Records(), policy=Policy())
calc_puf.advance_to_year(2014)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [30]:
# Calculator the CPS
wt = pd.read_csv('../Dropbox/cps_weights.csv')
# wt.drop('SEQUENCE', inplace=True, axis=1)
# wt = wt.loc[:,:'WT2026']
wt *= 100
recs_cps = Records(data=cps,
                   weights=wt,
                   adjust_ratios=None,
                   start_year=2014)
pol_cps = Policy(start_year=2014, num_years=11)
calc_cps = Calculator(records=recs_cps, policy=pol_cps)
calc_cps.advance_to_year(2014)

In [31]:
calc_puf.calc_all()
calc_cps.calc_all()

### CPS Distribution Table

In [32]:
create_distribution_table(calc_cps.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,-97,"16,319,161",-441,"9,074,074","6,081","15,563",6,"4,925",24,1,-445,0,0,1,0,0,87,-87,153,67
1,"8,731","16,319,712","4,716","11,322,765","7,316","145,610",88,"5,616",120,8,"4,680","13,390",0,8,2,0,706,-700,617,-83
2,"15,175","16,319,630","8,109","11,822,840","7,552","802,812",552,"5,991","1,288",121,"7,851","35,542",1,123,22,0,974,-873,"1,078",205
3,"21,737","16,319,485","14,319","12,933,032","7,601","1,943,638","1,165","6,664","4,025",409,"13,696","35,660",2,411,69,0,"1,173",-831,"1,949","1,119"
4,"29,156","16,318,339","21,672","12,619,234","7,568","3,087,437","1,947","7,190","8,516",974,"20,577","28,039",3,977,137,0,"1,022",-183,"3,002","2,819"
5,"38,596","16,320,857","30,661","12,520,710","7,713","3,592,013","2,530","7,630","15,087","1,797","29,317","22,506",3,"1,800",235,0,599,966,"4,248","5,214"
6,"51,645","16,319,826","44,131","11,893,453","7,492","4,344,541","3,558","8,007","25,981","3,277","42,332","25,293",3,"3,281",368,0,151,"2,762","6,063","8,825"
7,"71,129","16,319,281","64,412","10,979,130","7,220","5,335,131","5,053","8,783","43,687","6,190","62,120","14,427",2,"6,192",506,0,14,"5,673","8,832","14,504"
8,"102,785","16,319,888","94,688","8,386,083","5,917","7,931,146","7,847","9,926","71,157","10,816","91,647","50,441",7,"10,823",599,0,2,"10,222","13,114","23,337"
9,"246,886","16,319,593","233,402","5,230,997","3,857","11,088,139","14,357","9,762","205,492","47,126","229,642","2,949,970",668,"47,794",122,377,0,"48,049","21,905","69,954"


### PUF Distribution Table

In [33]:
create_distribution_table(calc_puf.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,904","16,222,114","-7,252","11,094,216","5,964","6,150",18,"4,135",44,4,"-7,601",0,0,4,0,6,141,-131,353,222
1,"8,443","16,220,773","6,016","13,627,903","7,057","48,948",29,"4,913",262,23,"5,990","20,950",0,24,0,4,774,-747,821,74
2,"15,025","16,223,499","10,098","13,243,287","7,608","279,753",180,"6,493","1,482",145,"9,974","27,189",1,146,19,6,"1,650","-1,517","1,428",-89
3,"21,963","16,222,202","15,120","14,169,419","7,787","683,754",548,"6,822","3,991",420,"14,767","31,485",2,422,82,13,"1,460","-1,107","2,039",932
4,"30,413","16,223,284","22,440","14,142,144","7,921","1,516,658","1,287","7,187","8,668",999,"21,647","18,616",2,"1,002",217,17,"1,067",-265,"2,881","2,615"
5,"40,936","16,222,555","32,564","13,206,631","7,687","2,863,096","2,661","7,339","16,532","2,017","30,986","15,840",2,"2,020",362,26,548,"1,136","3,965","5,101"
6,"54,706","16,222,223","46,478","11,339,850","7,067","4,849,894","5,036","7,734","27,243","3,573","43,467","10,617",2,"3,575",506,42,164,"2,947","5,336","8,283"
7,"74,668","16,222,702","67,782","9,028,824","6,082","7,185,021","8,156","8,574","45,098","6,519","63,224","46,267",5,"6,524",681,62,80,"5,825","7,708","13,533"
8,"110,558","16,222,104","102,839","5,871,767","4,308","10,349,258","13,730","10,142","74,728","11,702","95,552","184,541",18,"11,720",889,100,95,"10,835","12,479","23,314"
9,"338,763","16,223,164","321,658","2,019,136","1,497","14,190,613","36,479","9,580","274,512","69,165","306,721","4,448,053","2,351","71,516","1,789","2,138",55,"71,811","21,957","93,768"


### Diagnostic Table Comparison

In [34]:
cps_diag = create_diagnostic_table(calc_cps)

In [35]:
puf_diag = create_diagnostic_table(calc_puf)

In [36]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['CPS'] = cps_diag[2014]
diag_data['PUF'] = puf_diag[2014]
diag_data['% Change'] = ((cps_diag[2014] / puf_diag[2014]) - 1) * 100

In [37]:
diag_data

,SOI,CPS,PUF,% Change
Returns (#m),148.6,163.2,162.2,0.6
AGI ($b),"9,771.00","8,415.6","10,021.5",-16.0
Itemizers (#m),44,38.3,42.0,-8.8
Itemized Deduction ($b),1206.7,605.5,"1,105.2",-45.2
Standard Deduction Filers (#m),117.4,106.8,107.7,-0.9
Standard Deduction ($b),876.2,966.4,925.2,4.4
Personal Exemption ($b),1121.6,"1,105.4","1,114.0",-0.8
Taxable Income ($b),6997.9,"6,126.0","7,341.8",-16.6
Regular Tax ($b),NaN,"1,154.1","1,534.2",-24.8
AMT Income ($b),NaN,"8,182.9","9,485.9",-13.7


In [38]:
# Total value of missing itemized deductions
in_billions = 1e-9
state = (calc_puf.records.e18400 * calc_puf.records.s006).sum()
print 'State and Local: {} ($b)'.format(round(state * in_billions, 2))
int_paid = (calc_puf.records.e19200 * calc_puf.records.s006).sum()
print 'Interest Paid: {} ($b)'.format(round(int_paid * in_billions, 2))
net_cas = (calc_puf.records.g20500 * calc_puf.records.s006).sum()
print 'Net Casualty or Theft Loss: {} ($b)'.format(round(net_cas * in_billions, 2))
print '-----------------'
print 'Total: {} ($b)'.format((round((state + int_paid + net_cas) * in_billions, 2)))

State and Local: 321.94 ($b)
Interest Paid: 325.15 ($b)
Net Casualty or Theft Loss: 4.62 ($b)
-----------------
Total: 651.72 ($b)


### Income Levels

In [39]:
inc_dict = OrderedDict()
inc_dict['CPS'] = [] 
inc_dict['PUF'] = []
inc_dict['Diff'] = []
inc_dict['Pct Diff'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
was_cps = (calc_cps.records.e00200 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(was_cps)
was_puf = (calc_puf.records.e00200 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(was_puf)
was_diff = (was_cps - was_puf)
inc_dict['Diff'].append(was_diff)
inc_dict['Pct Diff'].append((was_diff / was_puf) * 100)

int_cps = (calc_cps.records.e00300 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(int_cps)
int_puf = (calc_puf.records.e00300 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(int_puf)
int_diff = (int_cps - int_puf)
inc_dict['Diff'].append(int_diff)
inc_dict['Pct Diff'].append((int_diff / int_puf) * 100)

odiv_cps = (calc_cps.records.e00600 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(odiv_cps)
odiv_puf = (calc_puf.records.e00600 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(odiv_puf)
odiv_diff = (odiv_cps - odiv_puf)
inc_dict['Diff'].append(odiv_diff)
inc_dict['Pct Diff'].append((odiv_diff / odiv_puf) * 100)

qdiv_cps = (calc_cps.records.e00650 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(qdiv_cps)
qdiv_puf = (calc_puf.records.e00650 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(qdiv_puf)
qdiv_diff = (qdiv_cps - qdiv_puf)
inc_dict['Diff'].append(qdiv_diff)
inc_dict['Pct Diff'].append((qdiv_diff / qdiv_puf) * 100)

biz_cps = (calc_cps.records.e00900 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(biz_cps)
biz_puf = (calc_puf.records.e00900 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(biz_puf)
biz_diff = (biz_cps - biz_puf)
inc_dict['Diff'].append(biz_diff)
inc_dict['Pct Diff'].append((biz_diff / biz_puf) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
inc_df

,CPS,PUF,Diff,Pct Diff
WAS,"6,649,181,868,021.7","6,820,912,371,695.9","-171,730,503,674.2",-2.5
Taxable Interest,"92,043,813,870.5","97,246,196,658.4","-5,202,382,787.9",-5.3
Ordinary Dividends,"246,375,676,261.9","270,137,086,776.2","-23,761,410,514.2",-8.8
Qualified Dividends,"186,161,460,983.5","198,124,994,913.5","-11,963,533,929.9",-6.0
Business Income,"310,777,040,492.4","309,444,223,964.1","1,332,816,528.4",0.4


### Distribution of Income Variables

In [40]:
# Generate data for distribution plots
cps_dist = pd.DataFrame()
puf_dist = pd.DataFrame()
cps_was = distribution(calc_cps.records.e00200, calc_cps.records.s006, calc_cps.records.c00100)
puf_was = distribution(calc_puf.records.e00200, calc_puf.records.s006, calc_puf.records.c00100)
cps_int = distribution(calc_cps.records.e00300, calc_cps.records.s006, calc_cps.records.c00100)
puf_int = distribution(calc_puf.records.e00300, calc_puf.records.s006, calc_puf.records.c00100)
cps_odiv = distribution(calc_cps.records.e00600, calc_cps.records.s006, calc_cps.records.c00100)
puf_odiv = distribution(calc_puf.records.e00600, calc_puf.records.s006, calc_puf.records.c00100)
cps_qdiv = distribution(calc_cps.records.e00650, calc_cps.records.s006, calc_cps.records.c00100)
puf_qdiv = distribution(calc_puf.records.e00650, calc_puf.records.s006, calc_puf.records.c00100)
cps_biz = distribution(calc_cps.records.e00900, calc_cps.records.s006, calc_cps.records.c00100)
puf_biz = distribution(calc_puf.records.e00900, calc_puf.records.s006, calc_puf.records.c00100)
cps_dist['WAS'] = cps_was[1]
puf_dist['WAS'] = puf_was[1]
cps_dist['INT'] = cps_int[1]
puf_dist['INT'] = puf_int[1]
cps_dist['ODIV'] = cps_odiv[1]
puf_dist['ODIV'] = puf_odiv[1]
cps_dist['QDIV'] = cps_qdiv[1]
puf_dist['QDIV'] = puf_qdiv[1]
cps_dist['BIZ'] = cps_biz[1]
puf_dist['BIZ'] = puf_biz[1]
cps_dist['AGI Bin'] = index_list()
puf_dist['AGI Bin'] = index_list()
cps_dist['label'] = 'CPS'
puf_dist['label'] = 'PUF'
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum()) * 100
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'
scatter_data = pd.concat([cps_dist, puf_dist, soi_dist])
scatter_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(scatter_data, x='AGI Bin', y=item[0], color='label', ylabel='Percent',
                      title=title, tooltips=[('PCT', '@{}'.format(item[0]))])
    scatter_list.append(scatter)

In [41]:
show(column(scatter_list))

In [42]:
cps_tot = pd.DataFrame()
puf_tot = pd.DataFrame()
cps_tot['WAS'] = cps_was[0]
puf_tot['WAS'] = puf_was[0]
cps_tot['INT'] = cps_int[0]
puf_tot['INT'] = puf_int[0]
cps_tot['ODIV'] = cps_odiv[0]
puf_tot['ODIV'] = puf_odiv[0]
cps_tot['QDIV'] = cps_qdiv[0]
puf_tot['QDIV'] = puf_qdiv[0]
cps_tot['BIZ'] = cps_biz[0]
puf_tot['BIZ'] = puf_biz[0]
cps_tot['AGI Bin'] = index_list()
puf_tot['AGI Bin'] = index_list()
cps_tot['label'] = 'CPS'
puf_tot['label'] = 'PUF'
soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'
total_data = pd.concat([cps_tot, puf_tot, soi_income])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
total_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(total_data, x='AGI Bin', y=item[0], color='label', ylabel='Total',
                      title=title, tooltips=[('Total', '@{}'.format(item[0]))])
    total_list.append(scatter)

In [43]:
show(column(total_list))

### Itemized Deduction Amounts

In [44]:
deductions_cps = {'Medical Expenses': (calc_cps.records.e17500[calc_cps.records.c04470 > 0] *
                                       calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'State and Local Taxes':  (calc_cps.records.e18400[calc_cps.records.c04470 > 0] *
                                             calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Real Estate Taxes':  (calc_cps.records.e18500[calc_cps.records.c04470 > 0] *
                                         calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Interest Paid':  (calc_cps.records.e19200[calc_cps.records.c04470 > 0] *
                                     calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Charitable Cash Contributions': (calc_cps.records.e19800[calc_cps.records.c04470 > 0] *
                                                    calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Charitable Non-Cash Contributions': (calc_cps.records.e20100[calc_cps.records.c04470 > 0] *
                                                        calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Total Misc. Expenses':  (calc_cps.records.e20400[calc_cps.records.c04470 > 0] *
                                            calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Net Casualty or Loss': (calc_cps.records.g20500[calc_cps.records.c04470 > 0] *
                                           calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum()}
ded_cps_df = pd.DataFrame.from_dict(deductions_cps, 'index')
ded_cps_df.columns = ['Total']
ded_cps_df['source'] = 'CPS'

deductions_puf = {'Medical Expenses': (calc_puf.records.e17500[calc_puf.records.c04470 > 0] *
                                       calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'State and Local Taxes':  (calc_puf.records.e18400[calc_puf.records.c04470 > 0] *
                                             calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Real Estate Taxes':  (calc_puf.records.e18500[calc_puf.records.c04470 > 0] *
                                         calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Interest Paid':  (calc_puf.records.e19200[calc_puf.records.c04470 > 0] *
                                     calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Charitable Cash Contributions': (calc_puf.records.e19800[calc_puf.records.c04470 > 0] *
                                                    calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Charitable Non-Cash Contributions': (calc_puf.records.e20100[calc_puf.records.c04470 > 0] *
                                                        calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Total Misc. Expenses':  (calc_puf.records.e20400[calc_puf.records.c04470 > 0] *
                                            calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Net Casualty or Loss': (calc_puf.records.g20500[calc_puf.records.c04470 > 0] *
                                           calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum()}
ded_puf_df = pd.DataFrame.from_dict(deductions_puf, 'index')
ded_puf_df.columns = ['Total']
ded_puf_df['source'] = 'PUF'

ded_full_df = pd.concat([ded_cps_df, ded_puf_df])
ded_full_df['index'] = ded_full_df.index

In [45]:
ded_bar = Bar(ded_full_df, 'index', 'Total', group='source', title='Itemized Deduction Totals',
              xlabel='Deduction', ylabel='Total', tooltips=[('Deduction', '@index'), ('Total', '@height'),
                                                            ('Data', '@source')])
show(ded_bar)

In [46]:
eitc_cps = (calc_cps.records.eitc * calc_cps.records.s006).sum()
c11070_cps = (calc_cps.records.c11070 * calc_cps.records.s006).sum()
c10960_cps = (calc_cps.records.c10960 * calc_cps.records.s006).sum()
personal_credit_cps = (calc_cps.records.personal_credit * calc_cps.records.s006).sum()
ctc_new_cps = (calc_cps.records.ctc_new * calc_cps.records.s006).sum()

eitc_puf = (calc_puf.records.eitc * calc_puf.records.s006).sum()
c11070_puf = (calc_puf.records.c11070 * calc_puf.records.s006).sum()
c10960_puf = (calc_puf.records.c10960 * calc_puf.records.s006).sum()
personal_credit_puf = (calc_puf.records.personal_credit * calc_puf.records.s006).sum()
ctc_new_puf = (calc_puf.records.ctc_new * calc_puf.records.s006).sum()

In [54]:
print 'Refundable Credits'
pd.DataFrame(OrderedDict({'PUF': [eitc_puf, c11070_puf, c10960_puf, personal_credit_puf, ctc_new_puf],
                          'CPS': [eitc_cps, c11070_cps, c10960_cps, personal_credit_cps, ctc_new_cps],
                          'Diff': [eitc_cps - eitc_puf, c11070_cps - c11070_puf, c10960_cps - c10960_puf,
                                   personal_credit_cps - personal_credit_puf,
                                   ctc_new_cps - ctc_new_puf]}),
                          index=['eitc', 'c11070', 'c10960', 'personal credit', 'ctc new'])

Refundable Credits


,Diff,PUF,CPS
eitc,"-6,968,041,838.7","65,207,453,506.9","58,239,411,668.3"
c11070,"-5,342,915,580.0","24,274,900,731.4","18,931,985,151.4"
c10960,"-8,382,574,969.7","8,382,574,969.7",0.0
personal credit,0.0,0.0,0.0
ctc new,0.0,0.0,0.0


In [48]:
print 'EITC - CPS: {}'.format((calc_cps.records.EIC * calc_cps.records.s006).sum())
print 'EITC - PUF: {}'.format((calc_puf.records.EIC * calc_puf.records.s006).sum())

EITC - CPS: 82564314.0867
EITC - PUF: 35762859.59


In [49]:
eic0cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 0])
eic1cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 1])
eic2cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 2])
eic3cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 3])
eic0puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 0])
eic1puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 1])
eic2puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 2])
eic3puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 3]) 
eic_tot_cps = calc_cps.records.s006.sum()
eic_tot_puf = calc_puf.records.s006.sum()
pd.DataFrame({'CPS': [eic0cps, eic1cps, eic2cps, eic3cps],
              'PUF': [eic0puf, eic1puf, eic2puf, eic3puf],
              '% - CPS': [eic0cps / eic_tot_cps, eic1cps / eic_tot_cps,
                          eic2cps / eic_tot_cps, eic3cps / eic_tot_cps],
              '% - PUF': [eic0puf / eic_tot_puf, eic1puf / eic_tot_puf,
                          eic2puf / eic_tot_puf, eic3puf / eic_tot_puf]})

,% - CPS,% - PUF,CPS,PUF
0,0.7,0.9,"114,745,569.5","140,952,059.1"
1,0.1,0.1,"22,983,420.2","10,061,670.4"
2,0.1,0.0,"16,819,448.2","7,931,481.3"
3,0.1,0.0,"8,647,332.5","3,279,408.9"
